# 実践演習 Day 1：streamlitとFastAPIのデモ
このノートブックでは以下の内容を学習します。

- 必要なライブラリのインストールと環境設定
- Hugging Faceからモデルを用いたStreamlitのデモアプリ
- FastAPIとngrokを使用したAPIの公開方法

演習を始める前に、HuggingFaceとngrokのアカウントを作成し、
それぞれのAPIトークンを取得する必要があります。


演習の時間では、以下の3つのディレクトリを順に説明します。

1. 01_streamlit_UI
2. 02_streamlit_app
3. 03_FastAPI

2つ目や3つ目からでも始められる様にノートブックを作成しています。

復習の際にもこのノートブックを役立てていただければと思います。

### 注意事項
「02_streamlit_app」と「03_FastAPI」では、GPUを使用します。

これらを実行する際は、Google Colab画面上のメニューから「編集」→ 「ノートブックの設定」

「ハードウェアアクセラレーター」の項目の中から、「T4 GPU」を選択してください。

このノートブックのデフォルトは「CPU」になっています。

---

# 環境変数の設定（1~3共有）


GitHubから演習用のコードをCloneします。

In [ ]:
# !git clone https://github.com/matsuolab/lecture-ai-engineering.git

必要なAPIトークンを.envに設定します。

「lecture-ai-engineering/day1」の配下に、「.env_template」ファイルが存在しています。

隠しファイルのため表示されていない場合は、画面左側のある、目のアイコンの「隠しファイルの表示」ボタンを押してください。

「.env_template」のファイル名を「.env」に変更します。「.env」ファイルを開くと、以下のような中身になっています。


```
HUGGINGFACE_TOKEN="hf-********"
NGROK_TOKEN="********"
```
ダブルクオーテーションで囲まれた文字列をHuggingfaceのアクセストークンと、ngrokの認証トークンで書き変えてください。

それぞれのアカウントが作成済みであれば、以下のURLからそれぞれのトークンを取得できます。

- Huggingfaceのアクセストークン
https://huggingface.co/docs/hub/security-tokens

- ngrokの認証トークン
https://dashboard.ngrok.com/get-started/your-authtoken

書き換えたら、「.env」ファイルをローカルのPCにダウンロードしてください。

「01_streamlit_UI」から「02_streamlit_app」へ進む際に、CPUからGPUの利用に切り替えるため、セッションが一度切れてしまいます。

その際に、トークンを設定した「.env」ファイルは再作成することになるので、その手間を減らすために「.env」ファイルをダウンロードしておくと良いです。

「.env」ファイルを読み込み、環境変数として設定します。次のセルを実行し、最終的に「True」が表示されていればうまく読み込めています。

In [1]:
# !pip install python-dotenv
from dotenv import load_dotenv, find_dotenv

# %cd /content/lecture-ai-engineering/day1
load_dotenv(find_dotenv())

True

環境変数を一時的に保存しておくコード

In [3]:
import os
import json

def save_env_backup():
        env_backup = {
                "HUGGINGFACE_TOKEN": os.environ.get("HUGGINGFACE_TOKEN"),
                "NGROK_TOKEN": os.environ.get("NGROK_TOKEN")
            }
        with open('/content/env_backup.json', 'w') as f:
              json.dump(env_backup, f)
        print("環境変数のバックアップを作成しました。セッションが切断された場合は restore_env_backup() を実行してください。")

        # 環境変数を復元する関数
        def restore_env_backup():
            try:
                with open('/content/env_backup.json', 'r') as f:
                    env_backup = json.load(f)

                # 環境変数を復元
                for key, value in env_backup.items():
                    os.environ[key] = value

                # .env ファイルを再作成\n",
                with open('/content/lecture-ai-engineering/day1/.env', 'w') as f:
                    for key, value in env_backup.items():
                        f.write(f"{key}={value}")

                print("環境変数を復元しました。")
                return True
            except FileNotFoundError:
                print("バックアップファイルが見つかりません。環境変数を手動で設定してください。")
                return False

        # バックアップを作成
        save_env_backup()

# 01_streamlit_UI

ディレクトリ「01_streamlit_UI」に移動します。

In [4]:
%cd /content/lecture-ai-engineering/day1/01_streamlit_UI

/content/lecture-ai-engineering/day1/01_streamlit_UI


必要なライブラリをインストールします。

In [5]:
%%capture
!pip install -r requirements.txt

# 追加ライブラリのインストール
!pip install streamlit-option-menu streamlit-authenticator

**新しい UI コンポーネントを追加**

In [56]:
%%writefile custom_ui.py
"""
custom_ui.py
-----------------
Streamlit 用のサイドバー＆カスタムページ部品。
• メニュー選択・テーマ切替
• ホームページのカードとプログレスバー
"""
from __future__ import annotations

import time
import streamlit as st
from streamlit_option_menu import option_menu


# ----------------------------------------------------------------------
# サイドバー
# ----------------------------------------------------------------------
_MENU_ITEMS = ["ホーム", "基本要素", "レイアウト", "入力要素", "テーマ設定"]
_ICONS = ["house", "list-task", "columns", "input-cursor", "palette"]


def _apply_dark_theme(enable_dark: bool) -> None:
    """ダークテーマを CSS で適用／解除する。"""
    if enable_dark:
        st.markdown(
            """
            <style>
            .main {background-color: #0E1117; color: #FFFFFF;}
            .sidebar .sidebar-content {background-color: #262730; color: #FFFFFF;}
            </style>
            """,
            unsafe_allow_html=True,
        )


def create_sidebar() -> str:
    """
    カスタムサイドバーを生成し、選択されたメニューを返す。

    Returns
    -------
    str
        現在選択中のページ名。
    """
    with st.sidebar:
        selected = option_menu(
            menu_title="メインメニュー",
            options=_MENU_ITEMS,
            icons=_ICONS,
            menu_icon="cast",
            default_index=0,
        )

        # テーマ切替スイッチ
        if "light_mode" not in st.session_state:
            st.session_state.light_mode = True

        if st.button("🌓 テーマ切替"):
            st.session_state.light_mode = not st.session_state.light_mode

    # サイドバー外でテーマ適用（CSS は 1 度のみ挿入）
    _apply_dark_theme(not st.session_state.light_mode)

    return selected


# ----------------------------------------------------------------------
# 共通ユーティリティ
# ----------------------------------------------------------------------
def page_hourglass() -> None:
    """デモ用のプログレスバーを表示するページ。"""
    st.subheader("改善されたプログレス表示")
    progress_text = "処理中です。しばらくお待ちください..."
    bar = st.progress(0, text=progress_text)
    placeholder = st.empty()

    for percent in range(100):
        time.sleep(0.02)
        bar.progress(percent + 1, text=f"{progress_text} ({percent + 1}%)")
        if percent % 10 == 0:
            placeholder.info(f"Step {percent // 10 + 1}/10 完了")

    bar.empty()
    placeholder.empty()
    st.success("処理が完了しました！")


# ----------------------------------------------------------------------
# ページ切替ハブ
# ----------------------------------------------------------------------
def show_custom_pages(selected: str) -> str:
    """
    選択されたページに対応するコンテンツを描画する。

    Parameters
    ----------
    selected : str
        create_sidebar から返されたページ名。

    Returns
    -------
    str
        同じ値を返すだけ（呼び出し元での再利用用）。
    """
    if selected == "ホーム":
        _render_home()

    # そのほかのページは app.py で処理
    return selected


def _render_home() -> None:
    """ホームページを描画。"""
    st.title("Streamlit UIデモ（改善版）")
    st.write(
        """
        このデモアプリは、Streamlit の基本的な UI 要素を紹介するものです。
        サイドバーから異なるセクションを選択して、さまざまなコンポーネントをお試しください。
        """
    )

    # カード風に 3 カラムで機能案内
    col1, col2, col3 = st.columns(3)
    with col1:
        st.info("**基本要素**\n\nテキスト、ヘッダー、メディアなど")
    with col2:
        st.success("**レイアウト**\n\n列、タブ、エキスパンダーなど")
    with col3:
        st.warning("**入力要素**\n\nボタン、スライダー、テキスト入力など")

    # プログレスバーのデモ起動
    if st.button("プログレスバーデモを表示"):
        page_hourglass()

Overwriting custom_ui.py


ngrokのトークンを使用して、認証を行います。

In [57]:
!ngrok authtoken $$NGROK_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


**app.pyの書き換え**

In [58]:
%%writefile app.py
"""Streamlit UI Demo — cleaned & refactored
------------------------------------------------
A single‑file demo app showcasing basic Streamlit components.
Split into small render_* functions for readability.
"""
from __future__ import annotations

import io
from datetime import date as dt_date
from typing import Callable, Dict

import altair as alt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import streamlit as st

from custom_ui import create_sidebar, show_custom_pages

# ----------------------------------------------------------------------------
# ページ共通ユーティリティ
# ----------------------------------------------------------------------------

def random_df(rows: int = 20, cols: int = 3, colnames: list[str] | None = None) -> pd.DataFrame:
    """ランダム DataFrame を生成。"""
    if colnames is None:
        colnames = list("ABC")[:cols]
    return pd.DataFrame(np.random.randn(rows, cols), columns=colnames)


# ----------------------------------------------------------------------------
# 個別ページ描画関数
# ----------------------------------------------------------------------------

def render_basic() -> None:
    """基本 UI 要素ページ"""
    st.title("基本要素")

    # --- テキスト類 ---
    st.header("テキスト要素")
    st.text("これは通常のテキストです。")
    st.markdown("**これはマークダウンテキストです。** *イタリック* や `コード` も使えます。")
    st.info("これは情報メッセージです。")
    st.warning("これは警告メッセージです。")
    st.error("これはエラーメッセージです。")
    st.success("これは成功メッセージです。")

    # --- メディア要素 ---
    st.header("メディア要素")
    tab_chart, tab_df, tab_img = st.tabs(["📈 チャート", "🗃 DataFrame", "🖼 画像"])

    with tab_chart:
        st.subheader("折れ線グラフ")
        chart_data = random_df()
        st.line_chart(chart_data)

        st.subheader("Altair チャート")
        c = (
            alt.Chart(chart_data.reset_index())
            .mark_circle()
            .encode(x="index", y="A", size="B", color="C", tooltip=["A", "B", "C"])
            .interactive()
        )
        st.altair_chart(c, use_container_width=True)

    with tab_df:
        st.subheader("DataFrame の表示")
        df = pd.DataFrame({
            "名前": ["Alice", "Bob", "Charlie", "David"],
            "年齢": [24, 42, 18, 31],
            "都市": ["東京", "大阪", "京都", "名古屋"],
        })
        st.dataframe(df, use_container_width=True)
        st.download_button("CSVとしてダウンロード", df.to_csv(index=False, encoding="utf-8-sig"), "sample_data.csv", "text/csv")

    with tab_img:
        st.subheader("動的に生成した画像")
        fig, ax = plt.subplots()
        x = np.linspace(0, 10, 100)
        ax.plot(x, np.sin(x))
        ax.set_title("サイン波")
        ax.grid(True)
        buf = io.BytesIO()
        fig.savefig(buf, format="png")
        st.image(buf.getvalue(), caption="動的に生成されたサイン波", use_column_width=True)


def render_layout() -> None:
    """レイアウト要素ページ"""
    st.title("レイアウト要素")

    st.header("カラムレイアウト")
    col1, col2, col3 = st.columns(3)
    with col1:
        st.subheader("カラム1")
        st.image("https://via.placeholder.com/150", caption="プレースホルダー画像")
    with col2:
        st.subheader("カラム2")
        st.metric("温度", "28°C", "1.2°C")
    with col3:
        st.subheader("カラム3")
        st.metric("湿度", "65%", "-4%", delta_color="inverse")

    st.header("エキスパンダー")
    with st.expander("詳細を表示"):
        st.write("""
            エキスパンダーを使用すると、長いコンテンツを折りたたむことができます。
            ユーザーが必要なときに展開できるため、画面スペースを節約できます。
        """)
        st.image("https://via.placeholder.com/400x200", caption="大きなプレースホルダー画像")

    st.header("タブ")
    tab1, tab2, tab3 = st.tabs(["Tab 1", "Tab 2", "Tab 3"])
    with tab1:
        st.bar_chart(random_df(10, 3, ["X", "Y", "Z"]))
    with tab2:
        st.line_chart(pd.DataFrame(np.sin(np.linspace(0, 10, 100))))
    with tab3:
        st.area_chart(random_df(10).cumsum())


def render_inputs() -> None:
    """入力要素ページ"""
    st.title("入力要素")

    # --- ボタン ---
    st.header("ボタン")
    if st.button("クリックしてください"):
        st.success("ボタンがクリックされました！")

    # --- チェックボックス ---
    st.header("チェックボックス")
    if st.checkbox("チェックボックスを表示"):
        st.write("チェックボックスがオンになっています。")

    # --- ラジオボタン ---
    st.header("ラジオボタン")
    genre = st.radio("好きな音楽ジャンルは？", ("ロック", "ポップ", "ジャズ", "クラシック"))
    if genre:
        st.write(f"あなたは {genre} を選択しました。")

    # --- セレクトボックス ---
    st.header("セレクトボックス")
    color = st.selectbox("好きな色は？", ("赤", "青", "緑", "黄色"))
    st.write(f"あなたが選んだのは: {color}")

    # --- マルチセレクト ---
    st.header("マルチセレクト")
    fruits = st.multiselect(
        "好きな果物は？",
        ["りんご", "バナナ", "オレンジ", "ぶどう", "いちご"],
        default=["りんご", "バナナ"],
    )
    st.write("あなたが選んだのは: " + ", ".join(fruits))

    # --- スライダー ---
    st.header("スライダー")
    age = st.slider("あなたの年齢は？", 0, 100, 25)
    st.write(f"あなたの年齢: {age}歳")

    # --- 範囲スライダー ---
    values = st.slider("値の範囲を選択:", 0.0, 100.0, (25.0, 75.0))
    st.write(f"選択された範囲: {values[0]} から {values[1]}")

    # --- 日付入力 ---
    st.header("日付入力")
    birth = st.date_input("生年月日を選択してください", dt_date(2000, 1, 1))
    st.write(f"あなたの生年月日: {birth}")

    # --- ファイルアップローダー ---
    st.header("ファイルアップローダー")
    uploaded = st.file_uploader("ファイルを選択してください", type=["csv", "xlsx", "txt", "jpg", "png"])
    if uploaded is not None:
        st.write(f"ファイル名: {uploaded.name} — {uploaded.size} bytes")
        if uploaded.type.startswith("image"):
            st.image(uploaded, caption="アップロードされた画像", use_column_width=True)
        elif uploaded.type == "text/plain":
            string_data = io.StringIO(uploaded.getvalue().decode()).read()
            st.text_area("ファイルの内容", string_data, height=200)
        elif uploaded.type == "text/csv":
            st.dataframe(pd.read_csv(uploaded), use_container_width=True)


# ----------------------------------------------------------------------------
# テーマ設定ページは custom_ui.py 内の CSS で制御
# ----------------------------------------------------------------------------

# ----------------------------------------------------------------------------
# ページ設定 & ルーティング
# ----------------------------------------------------------------------------

def main() -> None:
    st.set_page_config(
        page_title="Streamlit UIデモ（改善版）",
        page_icon="🧊",
        layout="wide",
        initial_sidebar_state="expanded",
    )

    selected = create_sidebar()
    selected = show_custom_pages(selected)  # ホームなど custom_ui 側

    page_table: Dict[str, Callable[[], None]] = {
        "基本要素": render_basic,
        "レイアウト": render_layout,
        "入力要素": render_inputs,
    }

    if selected in page_table:
        page_table[selected]()


if __name__ == "__main__":
    main()

Overwriting app.py


アプリを起動します。

In [59]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://f89e-34-125-210-255.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.210.255:8501

/content/lecture-ai-engineering/day1/01_streamlit_UI/app.py:84: UserWarning: Glyph 12469 (\N{KATAKANA LETTER SA}) missing from font(s) DejaVu Sans.
  fig.savefig(buf, format="png")
/content/lecture-ai-engineering/day1/01_streamlit_UI/app.py:84: UserWarning: Glyph 12452 (\N{KATAKANA LETTER I}) missing from font(s) DejaVu Sans.
  fig.savefig(buf, format="png")
/content/lecture-ai-engineering/day1/01_streamlit_UI/app.py:84: UserWarning: Glyph 12531 (\N{KATAKANA LETTER N}) missing from font(s) DejaVu Sans.
  fig.savefig(buf, format="png")
/content/lecture-ai-engineering/day1/01_streamlit_UI/app.py:84: UserWarning: Glyph 27874 (\N{CJK UNIFIED IDEOGRAPH-6CE2}) missing from font(s) DejaVu Sans.
  fig.savefig(buf, format="png")
2025-04-21 05:45:37.838 The `use_column_wid

公開URLの後に記載されているURLにブラウザでアクセスすると、streamlitのUIが表示されます。

app.pyのコメントアウトされている箇所を編集することで、UIがどの様に変化するか確認してみましょう。

streamlitの公式ページには、ギャラリーページがあります。

streamlitを使うとpythonという一つの言語であっても、様々なUIを実現できることがわかると思います。

https://streamlit.io/gallery

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [60]:
from pyngrok import ngrok
ngrok.kill()

# 02_streamlit_app


ディレクトリ「02_streamlit_app」に移動します。

In [61]:
%cd /content/lecture-ai-engineering/day1/02_streamlit_app

/content/lecture-ai-engineering/day1/02_streamlit_app


必要なライブラリをインストールします。

In [62]:
%%capture
!pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [63]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `AIE_2` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `AIE_2`


stramlitでHuggingfaceのトークン情報を扱うために、streamlit用の設定ファイル（.streamlit）を作成し、トークンの情報を格納します。

In [64]:
# .streamlit/secrets.toml ファイルを作成
import os
import toml

# 設定ファイルのディレクトリ確保
os.makedirs('.streamlit', exist_ok=True)

# 環境変数から取得したトークンを設定ファイルに書き込む
secrets = {
    "huggingface": {
        "token": os.environ.get("HUGGINGFACE_TOKEN", "")
    }
}

# 設定ファイルを書き込む
with open('.streamlit/secrets.toml', 'w') as f:
    toml.dump(secrets, f)

アプリを起動します。

02_streamlit_appでは、Huggingfaceからモデルをダウンロードするため、初回起動には2分程度時間がかかります。

この待ち時間を利用して、app.pyのコードを確認してみましょう。

**config.py の書き換え**

In [107]:
%%writefile config.py
# モデル設定
DATABASE_FILE = "chat_history.db"
MODEL_NAME = "meta-llama/Llama-3-8B-Japanese"

# UI設定
MAX_HISTORY_ITEMS = 10
THEME_COLOR = "#FF4B4B"
ENABLE_DARK_MODE = True
ENABLE_CACHING = True  # 追加：キャッシング機能の有効化
STREAM_OUTPUT = False  # ストリーミング出力を無効化

# 評価指標設定
METRICS = [
    "bleu_score",
    "cosine_similarity",
    "sentiment_score",
    "response_time",
    "token_generation_speed",
    "perplexity"
]

# 追加機能設定
ENABLE_CHAT_EXPORT = True
ENABLE_ERROR_LOGGING = True

Overwriting config.py


**metrics.py の書き換え**

In [90]:
%%writefile metrics.py
import time
import numpy as np
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textblob import TextBlob
import nltk
import math
import re

# 必要なNLTKデータのダウンロード
try:
    nltk.download('punkt', quiet=True)
    print("NLTK loaded successfully.")
except:
    pass

class MetricsCalculator:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()

    def calculate_bleu_score(self, reference, candidate):
        """BLEUスコアを計算 (0-1, 高いほど良い)"""
        reference_tokens = nltk.word_tokenize(reference.lower())
        candidate_tokens = nltk.word_tokenize(candidate.lower())
        return sentence_bleu([reference_tokens], candidate_tokens)

    def calculate_cosine_similarity(self, text1, text2):
        """コサイン類似度を計算 (0-1, 高いほど似ている)"""
        try:
            tfidf_matrix = self.vectorizer.fit_transform([text1, text2])
            return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
        except:
            return 0.0

    def calculate_sentiment_score(self, text):
        """感情分析スコアを計算 (-1: 非常にネガティブ, 1: 非常にポジティブ)"""
        sentiment = TextBlob(text).sentiment.polarity
        return sentiment

    def calculate_sentiment_metrics(self, text):
        """詳細な感情分析メトリクスを計算"""
        sentiment = TextBlob(text).sentiment

        return {
            "polarity": sentiment.polarity,  # -1.0 to 1.0
            "subjectivity": sentiment.subjectivity,  # 0.0 to 1.0
            "vader_compound": sentiment.polarity,
            "vader_negative": max(0, -sentiment.polarity),
            "vader_neutral": 1.0 - abs(sentiment.polarity),
            "vader_positive": max(0, sentiment.polarity)
        }

    def calculate_perplexity(self, text, n=3):
        """単純なN-gramモデルに基づくテキストの複雑さ推定（低いほど自然）"""
        tokens = nltk.word_tokenize(text.lower())
        if len(tokens) < n:
            return float('inf')  # テキストが短すぎる場合

        # N-gramカウント
        ngrams = {}
        for i in range(len(tokens) - n + 1):
            gram = ' '.join(tokens[i:i+n-1])
            next_token = tokens[i+n-1]

            if gram not in ngrams:
                ngrams[gram] = {}

            if next_token not in ngrams[gram]:
                ngrams[gram][next_token] = 0

            ngrams[gram][next_token] += 1

        # パープレキシティ計算
        log_prob = 0.0
        for i in range(len(tokens) - n + 1):
            gram = ' '.join(tokens[i:i+n-1])
            next_token = tokens[i+n-1]

            if gram in ngrams and next_token in ngrams[gram]:
                total = sum(ngrams[gram].values())
                prob = ngrams[gram][next_token] / total
                log_prob += math.log2(prob)
            else:
                log_prob += math.log2(1e-10)  # スムージング

        perplexity = 2 ** (-log_prob / (len(tokens) - n + 1))
        return perplexity

    def calculate_response_time(self, start_time):
        """応答時間を計算（秒単位）"""
        return time.time() - start_time

    def calculate_token_generation_speed(self, text, generation_time):
        """トークン生成速度を計算（トークン/秒）"""
        if generation_time <= 0:
            return 0
        # 簡易的なトークン数推定（実際にはモデルのトークナイザーによって異なる）
        token_count = len(re.findall(r'\w+|[^\w\s]', text))
        return token_count / generation_time

    def calculate_all_metrics(self, reference, candidate, generation_time):
        """すべての評価指標を計算"""
        metrics = {
            "bleu_score": self.calculate_bleu_score(reference, candidate),
            "cosine_similarity": self.calculate_cosine_similarity(reference, candidate),
            "sentiment": self.calculate_sentiment_metrics(candidate),
            "response_time": generation_time,
            "token_generation_speed": self.calculate_token_generation_speed(candidate, generation_time),
            "perplexity": self.calculate_perplexity(candidate)
        }
        return metrics

Overwriting metrics.py


**llm.pyの書き換え**

In [100]:
%%writefile llm.py
import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import time
from config import MODEL_NAME, STREAM_OUTPUT, ENABLE_CACHING

class LLMGenerator:
    def __init__(self):
        self.model = None
        self.tokenizer = None
        self.load_model()

    @st.cache_resource
    def load_model_cached(_self):
        """モデルをロードしてキャッシュする"""
        print(f"モデル {MODEL_NAME} をロード中...")

        # 量子化設定
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
        )

        # トークナイザーとモデルのロード
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

        # モデルのロード（4bit量子化）
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            quantization_config=quantization_config,
            device_map="auto",
            torch_dtype=torch.float16
        )

        return model, tokenizer

    def load_model(self):
        """モデルのロード処理（キャッシュが有効な場合はキャッシュを使用）"""
        if ENABLE_CACHING:
            self.model, self.tokenizer = self.load_model_cached()
        else:
            print(f"モデル {MODEL_NAME} をロード中...")

            # 量子化設定
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.float16
            )

            # トークナイザーとモデルのロード
            self.tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

            # モデルのロード（4bit量子化）
            self.model = AutoModelForCausalLM.from_pretrained(
                MODEL_NAME,
                quantization_config=quantization_config,
                device_map="auto",
                torch_dtype=torch.float16
            )

    def generate_text(self, prompt, max_length=512, temperature=0.7, stream_handler=None):
        """テキスト生成（シンプル版）"""
        start_time = time.time()

        try:
            # プロンプトのフォーマット
            if "meta-llama" in MODEL_NAME.lower():
                formatted_prompt = f"<human>: {prompt}\n<assistant>: "
            else:
                formatted_prompt = f"ユーザー: {prompt}\nシステム: "

            # 入力のエンコード
            inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to("cuda")

            generation_args = {
                "max_new_tokens": max_length,
                "temperature": temperature,
                "top_p": 0.95,
                "top_k": 50,
                "repetition_penalty": 1.1,
                "do_sample": temperature > 0.1,
                "pad_token_id": self.tokenizer.eos_token_id
            }

            # 生成
            output = self.model.generate(
                **inputs,
                **generation_args
            )

            # 出力のデコード
            generated_text = self.tokenizer.decode(output[0], skip_special_tokens=True)

            # プロンプト部分を削除
            generated_text = generated_text.replace(formatted_prompt, "")

            generation_time = time.time() - start_time
            return generated_text, generation_time

        except Exception as e:
            print(f"テキスト生成中にエラーが発生しました: {e}")
            generation_time = time.time() - start_time
            return f"エラーが発生しました: {e}", generation_time

Overwriting llm.py


**custom_ui.py 追加**

In [92]:
%%writefile custom_ui.py
import streamlit as st
from streamlit_option_menu import option_menu

def create_sidebar():
    """カスタムサイドバーを作成する関数"""
    with st.sidebar:
        selected = option_menu(
            "メインメニュー",
            ["ホーム", "基本要素", "レイアウト", "入力要素", "テーマ設定"],
            icons=["house", "list-task", "columns", "input-cursor", "palette"],
            menu_icon="cast",
            default_index=0,
        )

        # ダークモード/ライトモードの切り替え
        if "light_mode" not in st.session_state:
            st.session_state.light_mode = True

        if st.button("🌓 テーマ切替"):
            st.session_state.light_mode = not st.session_state.light_mode

        # カスタムCSS
        if not st.session_state.light_mode:
            st.markdown("""
            <style>
            .main {background-color: #0E1117; color: white;}
            .sidebar .sidebar-content {background-color: #262730; color: white;}
            </style>
            """, unsafe_allow_html=True)

        return selected

def page_hourglass():
    """プログレスバーデモページ"""
    import time

    st.subheader("改善されたプログレス表示")
    progress_text = "処理中です。しばらくお待ちください..."
    my_bar = st.progress(0, text=progress_text)
    placeholder = st.empty()

    for percent_complete in range(100):
        time.sleep(0.02)
        my_bar.progress(percent_complete + 1, text=f"{progress_text} ({percent_complete+1}%)")
        if percent_complete % 10 == 0:
            placeholder.info(f"Step {percent_complete // 10 + 1}/10 完了")

    my_bar.empty()
    placeholder.empty()
    st.success("処理が完了しました！")

def show_custom_pages(selected):
    """選択されたページに基づいてコンテンツを表示"""
    if selected == "ホーム":
        st.title("Streamlit UIデモ（改善版）")
        st.write("""このデモアプリは、Streamlitの基本的なUI要素を紹介するものです。
        サイドバーから異なるセクションを選択して、様々なStreamlitコンポーネントを試してみてください。""")

        # カード要素の追加
        col1, col2, col3 = st.columns(3)
        with col1:
            st.info("**基本要素**\n\nテキスト、ヘッダー、メディアなどの基本的なUI要素")
        with col2:
            st.success("**レイアウト**\n\n列、タブ、エキスパンダーなどのレイアウトオプション")
        with col3:
            st.warning("**入力要素**\n\nボタン、スライダー、テキスト入力などのインタラクティブ要素")

        # プログレスバーデモの表示
        if st.button("プログレスバーデモを表示"):
            page_hourglass()

    # 他のページの実装はapp.pyに任せる
    return selected

Overwriting custom_ui.py


**database.py 書き換え**

In [93]:
%%writefile database.py
import sqlite3
import json
import time
from datetime import datetime

class ChatDatabase:
    def __init__(self, db_file):
        self.db_file = db_file
        self.initialize_db()

    def initialize_db(self):
        """データベースとテーブルの初期化"""
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        # チャット履歴テーブル
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS chat_history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            session_id TEXT,
            timestamp TEXT,
            user_input TEXT,
            model_response TEXT,
            response_time REAL,
            metrics TEXT
        )
        ''')

        # フィードバックテーブル
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS feedback (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            chat_id INTEGER,
            rating INTEGER,
            feedback_text TEXT,
            timestamp TEXT,
            FOREIGN KEY (chat_id) REFERENCES chat_history (id)
        )
        ''')

        # エラーログテーブル（新規追加）
        cursor.execute('''
        CREATE TABLE IF NOT EXISTS error_logs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp TEXT,
            error_type TEXT,
            error_message TEXT,
            stack_trace TEXT,
            input_data TEXT
        )
        ''')

        conn.commit()
        conn.close()

    def save_chat(self, session_id, user_input, model_response, response_time, metrics=None):
        """チャット履歴を保存"""
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        timestamp = datetime.now().isoformat()
        metrics_json = json.dumps(metrics) if metrics else "{}"

        cursor.execute(
            "INSERT INTO chat_history (session_id, timestamp, user_input, model_response, response_time, metrics) VALUES (?, ?, ?, ?, ?, ?)",
            (session_id, timestamp, user_input, model_response, response_time, metrics_json)
        )

        chat_id = cursor.lastrowid
        conn.commit()
        conn.close()

        return chat_id

    def save_feedback(self, chat_id, rating, feedback_text=""):
        """フィードバックを保存"""
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        timestamp = datetime.now().isoformat()

        cursor.execute(
            "INSERT INTO feedback (chat_id, rating, feedback_text, timestamp) VALUES (?, ?, ?, ?)",
            (chat_id, rating, feedback_text, timestamp)
        )

        conn.commit()
        conn.close()

    def get_chat_history(self, session_id=None, limit=10, offset=0):
        """チャット履歴を取得"""
        conn = sqlite3.connect(self.db_file)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()

        if session_id:
            cursor.execute(
                "SELECT * FROM chat_history WHERE session_id = ? ORDER BY timestamp DESC LIMIT ? OFFSET ?",
                (session_id, limit, offset)
            )
        else:
            cursor.execute(
                "SELECT * FROM chat_history ORDER BY timestamp DESC LIMIT ? OFFSET ?",
                (limit, offset)
            )

        rows = cursor.fetchall()
        history = []

        for row in rows:
            feedback_cursor = conn.cursor()
            feedback_cursor.execute(
                "SELECT rating, feedback_text FROM feedback WHERE chat_id = ?",
                (row['id'],)
            )
            feedback = feedback_cursor.fetchone()

            chat_item = dict(row)
            if feedback:
                chat_item['feedback_rating'] = feedback['rating']
                chat_item['feedback_text'] = feedback['feedback_text']
            else:
                chat_item['feedback_rating'] = None
                chat_item['feedback_text'] = None

            try:
                chat_item['metrics'] = json.loads(chat_item['metrics'])
            except:
                chat_item['metrics'] = {}

            history.append(chat_item)

        conn.close()
        return history

    def get_total_chat_count(self, session_id=None):
        """チャット履歴の総数を取得"""
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        if session_id:
            cursor.execute(
                "SELECT COUNT(*) FROM chat_history WHERE session_id = ?",
                (session_id,)
            )
        else:
            cursor.execute("SELECT COUNT(*) FROM chat_history")

        count = cursor.fetchone()[0]
        conn.close()
        return count

    def log_error(self, error_type, error_message, stack_trace="", input_data=""):
        """エラーをログに記録（新規追加）"""
        conn = sqlite3.connect(self.db_file)
        cursor = conn.cursor()

        timestamp = datetime.now().isoformat()

        cursor.execute(
            "INSERT INTO error_logs (timestamp, error_type, error_message, stack_trace, input_data) VALUES (?, ?, ?, ?, ?)",
            (timestamp, error_type, error_message, stack_trace, input_data)
        )

        conn.commit()
        conn.close()

    def get_statistics(self):
        """チャット統計情報を取得（新規追加）"""
        conn = sqlite3.connect(self.db_file)
        conn.row_factory = sqlite3.Row
        cursor = conn.cursor()

        # 平均応答時間
        cursor.execute("SELECT AVG(response_time) as avg_response_time FROM chat_history")
        avg_response_time = cursor.fetchone()['avg_response_time'] or 0

        # 総チャット数
        cursor.execute("SELECT COUNT(*) as total_chats FROM chat_history")
        total_chats = cursor.fetchone()['total_chats'] or 0

        # 平均評価
        cursor.execute("SELECT AVG(rating) as avg_rating FROM feedback")
        avg_rating = cursor.fetchone()['avg_rating'] or 0

        # 評価分布
        cursor.execute("""
            SELECT rating, COUNT(*) as count
            FROM feedback
            GROUP BY rating
            ORDER BY rating
        """)
        rating_distribution = {row['rating']: row['count'] for row in cursor.fetchall()}

        conn.close()

        return {
            'avg_response_time': avg_response_time,
            'total_chats': total_chats,
            'avg_rating': avg_rating,
            'rating_distribution': rating_distribution
        }

# 使用例
if __name__ == "__main__":
    db = ChatDatabase("test.db")
    chat_id = db.save_chat("test_session", "こんにちは", "こんにちは！", 0.5, {"bleu_score": 0.8})
    db.save_feedback(chat_id, 5, "とても良い応答でした")
    history = db.get_chat_history("test_session")
    print(history)

Overwriting database.py


**app.py 書き換え**

In [110]:
%%writefile app.py
import streamlit as st
import uuid
import time
import json
from datetime import datetime

# ページ設定
st.set_page_config(
    page_title="改善版チャットアプリ",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded",
)

# セッションステートの初期化
if 'session_id' not in st.session_state:
    st.session_state.session_id = str(uuid.uuid4())
if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

# サイドバーの作成
with st.sidebar:
    st.title("改善版チャットアプリ")

    # テーマ設定
    theme = st.selectbox(
        "テーマ",
        ["ライト", "ダーク", "ブルー", "グリーン"]
    )

    # テーマに基づいてカスタムCSSを適用
    if theme == "ダーク":
        st.markdown("""
        <style>
        .main {background-color: #0E1117; color: white;}
        .stButton button {background-color: #4CAF50; color: white;}
        </style>
        """, unsafe_allow_html=True)
    elif theme == "ブルー":
        st.markdown("""
        <style>
        .main {background-color: #E8F4F8;}
        .stButton button {background-color: #0078D7; color: white;}
        </style>
        """, unsafe_allow_html=True)
    elif theme == "グリーン":
        st.markdown("""
        <style>
        .main {background-color: #E8F8E8;}
        .stButton button {background-color: #4CAF50; color: white;}
        </style>
        """, unsafe_allow_html=True)

    # メニュー選択
    page = st.radio(
        "メニュー",
        ["💬 チャット", "📚 履歴", "📊 分析"]
    )

    if page == "💬 チャット":
        st.subheader("設定")

        # 応答スタイル
        response_style = st.select_slider(
            "応答スタイル",
            options=["簡潔", "標準", "詳細"],
            value="標準"
        )

        # セッションリセット
        if st.button("新しい会話を開始"):
            st.session_state.chat_history = []
            st.session_state.session_id = str(uuid.uuid4())
            st.success("新しい会話を開始しました")

# チャットページ
if page == "💬 チャット":
    st.title("チャット")

    # チャット履歴の表示
    for chat in st.session_state.chat_history:
        # ユーザーメッセージ
        with st.chat_message("user"):
            st.write(chat["user_input"])

        # アシスタントメッセージ
        with st.chat_message("assistant"):
            st.write(chat["model_response"])

            # メトリクスの表示（折りたたみ可能）
            if "metrics" in chat:
                with st.expander("パフォーマンス指標"):
                    metrics = chat["metrics"]

                    # 基本メトリクス
                    cols = st.columns(3)
                    with cols[0]:
                        st.metric("応答時間", f"{metrics['response_time']:.2f}秒")
                    with cols[1]:
                        st.metric("文字数", f"{len(chat['model_response'])}")
                    with cols[2]:
                        st.metric("生成速度", f"{len(chat['model_response']) / metrics['response_time']:.1f} 文字/秒")

            # フィードバック
            if not chat.get("feedback_rating"):
                col1, col2, col3, col4, col5 = st.columns(5)
                with col3:
                    st.write("この回答はいかがでしたか？")

                col1, col2, col3 = st.columns(3)
                with col1:
                    if st.button("👎 悪い", key=f"bad_{chat['id']}"):
                        chat["feedback_rating"] = 1
                        st.experimental_rerun()
                with col2:
                    if st.button("👍 普通", key=f"fair_{chat['id']}"):
                        chat["feedback_rating"] = 3
                        st.experimental_rerun()
                with col3:
                    if st.button("👍👍 良い", key=f"good_{chat['id']}"):
                        chat["feedback_rating"] = 5
                        st.experimental_rerun()
            else:
                st.success(f"評価: {'👍' * (chat['feedback_rating'] // 2)}")

    # 新しいメッセージの入力
    user_input = st.chat_input("メッセージを入力してください")

    if user_input:
        # ユーザーメッセージの表示
        with st.chat_message("user"):
            st.write(user_input)

        # アシスタントメッセージの表示
        with st.chat_message("assistant"):
            message_placeholder = st.empty()

            # 応答生成（シミュレーション）
            start_time = time.time()
            with st.spinner("考え中..."):
                # 応答スタイルに基づいて異なる応答を生成
                if response_style == "簡潔":
                    time.sleep(0.5)
                    response = f"質問「{user_input}」への簡潔な回答です。"
                elif response_style == "詳細":
                    time.sleep(2.0)
                    response = f"""質問「{user_input}」への詳細な回答です。

                    ここではもっと詳しい説明を提供します。実際のLLMを使用した場合は、より関連性の高い情報が生成されます。

                    追加の詳細情報や例を含めることができます。このデモでは、実際のLLMの代わりに単純なテキスト生成を行っています。"""
                else:  # 標準
                    time.sleep(1.0)
                    response = f"質問「{user_input}」への回答です。実際のLLMを使用した場合は、より適切な回答が生成されます。このデモでは単純なテキスト生成を行っています。"

            end_time = time.time()
            gen_time = end_time - start_time

            message_placeholder.markdown(response)

            # メトリクスの計算
            metrics = {
                "response_time": gen_time,
                "text_length": len(response),
                "generation_speed": len(response) / gen_time
            }

            # チャット履歴をセッションに保存
            chat_id = len(st.session_state.chat_history)
            st.session_state.chat_history.append({
                "id": chat_id,
                "user_input": user_input,
                "model_response": response,
                "timestamp": datetime.now().isoformat(),
                "metrics": metrics
            })

            # フィードバックUI
            with st.expander("パフォーマンス指標"):
                cols = st.columns(3)
                with cols[0]:
                    st.metric("応答時間", f"{gen_time:.2f}秒")
                with cols[1]:
                    st.metric("文字数", f"{len(response)}")
                with cols[2]:
                    st.metric("生成速度", f"{len(response) / gen_time:.1f} 文字/秒")

            col1, col2, col3 = st.columns(3)
            with col1:
                if st.button("👎 悪い", key=f"bad_new"):
                    st.session_state.chat_history[-1]["feedback_rating"] = 1
                    st.experimental_rerun()
            with col2:
                if st.button("👍 普通", key=f"fair_new"):
                    st.session_state.chat_history[-1]["feedback_rating"] = 3
                    st.experimental_rerun()
            with col3:
                if st.button("👍👍 良い", key=f"good_new"):
                    st.session_state.chat_history[-1]["feedback_rating"] = 5
                    st.experimental_rerun()

# 履歴ページ
elif page == "📚 履歴":
    st.title("チャット履歴")

    if not st.session_state.chat_history:
        st.info("履歴がありません。チャットを始めましょう！")
    else:
        # CSV/JSONエクスポート機能
        col1, col2 = st.columns(2)

        with col1:
            # CSVデータの作成
            import io
            import pandas as pd

            csv_data = io.StringIO()
            history_df = pd.DataFrame([{
                'timestamp': item['timestamp'],
                'user_input': item['user_input'],
                'model_response': item['model_response'],
                'response_time': item['metrics']['response_time'],
                'rating': item.get('feedback_rating', 0)
            } for item in st.session_state.chat_history])

            history_df.to_csv(csv_data, index=False)

            st.download_button(
                label="CSVとしてエクスポート",
                data=csv_data.getvalue(),
                file_name=f"chat_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
                mime="text/csv"
            )

        with col2:
            # JSONエクスポート
            json_data = json.dumps([{
                'timestamp': item['timestamp'],
                'user_input': item['user_input'],
                'model_response': item['model_response'],
                'metrics': item['metrics'],
                'rating': item.get('feedback_rating', 0)
            } for item in st.session_state.chat_history], indent=2)

            st.download_button(
                label="JSONとしてエクスポート",
                data=json_data,
                file_name=f"chat_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
                mime="application/json"
            )

        # 履歴表示
        for i, item in enumerate(st.session_state.chat_history):
            with st.expander(f"{item['timestamp'][:19]} - {item['user_input'][:50]}..."):
                st.subheader("ユーザー入力")
                st.write(item['user_input'])

                st.subheader("モデル応答")
                st.write(item['model_response'])

                # メトリクス表示
                col1, col2, col3 = st.columns(3)

                with col1:
                    st.metric("応答時間", f"{item['metrics']['response_time']:.2f}秒")

                with col2:
                    st.metric("文字数", f"{len(item['model_response'])}")

                with col3:
                    if item.get('feedback_rating'):
                        st.metric("評価", f"{item['feedback_rating']}/5")
                    else:
                        st.info("評価なし")

# 分析ページ
elif page == "📊 分析":
    st.title("パフォーマンス分析")

    if not st.session_state.chat_history:
        st.info("まだデータがありません。チャットを始めて分析データを収集しましょう！")
    else:
        # 基本統計
        total_chats = len(st.session_state.chat_history)
        avg_response_time = sum(chat['metrics']['response_time'] for chat in st.session_state.chat_history) / total_chats
        ratings = [chat.get('feedback_rating', 0) for chat in st.session_state.chat_history if chat.get('feedback_rating')]
        avg_rating = sum(ratings) / len(ratings) if ratings else 0

        col1, col2, col3 = st.columns(3)

        with col1:
            st.metric("総チャット数", total_chats)

        with col2:
            st.metric("平均応答時間", f"{avg_response_time:.2f}秒")

        with col3:
            st.metric("平均評価", f"{avg_rating:.1f}/5" if ratings else "評価なし")

        # 応答時間の推移
        st.subheader("応答時間の推移")

        import pandas as pd
        import matplotlib.pyplot as plt

        time_data = pd.DataFrame([{
            'index': i,
            'response_time': chat['metrics']['response_time']
        } for i, chat in enumerate(st.session_state.chat_history)])

        st.line_chart(time_data.set_index('index')['response_time'])

        # 評価分布
        st.subheader("評価分布")

        from collections import Counter

        rating_counts = Counter(ratings)
        rating_df = pd.DataFrame({
            '評価': list(rating_counts.keys()),
            '回数': list(rating_counts.values())
        })

        if not rating_df.empty:
            fig, ax = plt.subplots()
            ax.bar(rating_df['評価'], rating_df['回数'])
            ax.set_xlabel('評価点数')
            ax.set_ylabel('回数')
            ax.set_xticks(range(1, 6))
            ax.grid(True, axis='y', linestyle='--', alpha=0.7)

            st.pyplot(fig)
        else:
            st.info("まだ評価データがありません。")

Overwriting app.py


In [111]:
from pyngrok import ngrok

public_url = ngrok.connect(8501).public_url
print(f"公開URL: {public_url}")
!streamlit run app.py

公開URL: https://612c-34-125-210-255.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.210.255:8501

  Stopping...


アプリケーションの機能としては、チャット機能や履歴閲覧があります。

これらの機能を実現するためには、StreamlitによるUI部分だけではなく、SQLiteを使用したチャット履歴の保存やLLMのモデルを呼び出した推論などの処理を組み合わせることで実現しています。

- **`app.py`**: アプリケーションのエントリーポイント。チャット機能、履歴閲覧、サンプルデータ管理のUIを提供します。
- **`ui.py`**: チャットページや履歴閲覧ページなど、アプリケーションのUIロジックを管理します。
- **`llm.py`**: LLMモデルのロードとテキスト生成を行うモジュール。
- **`database.py`**: SQLiteデータベースを使用してチャット履歴やフィードバックを保存・管理します。
- **`metrics.py`**: BLEUスコアやコサイン類似度など、回答の評価指標を計算するモジュール。
- **`data.py`**: サンプルデータの作成やデータベースの初期化を行うモジュール。
- **`config.py`**: アプリケーションの設定（モデル名やデータベースファイル名）を管理します。
- **`requirements.txt`**: このアプリケーションを実行するために必要なPythonパッケージ。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [112]:
from pyngrok import ngrok
ngrok.kill()

# 03_FastAPI

ディレクトリ「03_FastAPI」に移動します。

In [8]:
# %cd /content/lecture-ai-engineering/day1/03_FastAPI

必要なライブラリをインストールします。

In [ ]:
# %%capture
# !pip install -r requirements.txt

ngrokとhuggigfaceのトークンを使用して、認証を行います。

In [2]:
!ngrok authtoken $$NGROK_TOKEN
!huggingface-cli login --token $$HUGGINGFACE_TOKEN

Authtoken saved to configuration file: /Users/yamadayuuhei/Library/Application Support/ngrok/ngrok.yml
pyenv: invalid version `/Users/yamadayuuhei/Study/AIE/lecture-ai-engineering/.venv/bin/python' ignored in `/Users/yamadayuuhei/Study/AIE/lecture-ai-engineering/.python-version'
pyenv: invalid version `/Users/yamadayuuhei/Study/AIE/lecture-ai-engineering/.venv/bin/python' ignored in `/Users/yamadayuuhei/Study/AIE/lecture-ai-engineering/.python-version'
pyenv: huggingface-cli: command not found

The `huggingface-cli' command exists in these Python versions:
  3.12.4

Note: See 'pyenv help global' for tips on allowing both
      python2 and python3 to be found.


アプリを起動します。

「02_streamlit_app」から続けて「03_FastAPI」を実行している場合は、モデルのダウンロードが済んでいるため、すぐにサービスが立ち上がります。

「03_FastAPI」のみを実行している場合は、初回の起動時にモデルのダウンロードが始まるので、モデルのダウンロードが終わるまで数分間待ちましょう。

In [3]:
cd 03_FastAPI

/Users/yamadayuuhei/Study/AIE/lecture-ai-engineering/day1/03_FastAPI


In [6]:
!pip install transformers

  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
  Using cached PyYAML-6.0.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached regex-2024.11.6-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.5.3-cp38-abi3-macosx_11_0_arm64.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.1 MB/s eta 0:00:00
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached charset_normalizer-3.4.1-cp312-cp312-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 26.8 MB/s eta 0:00:00 

In [7]:
!python app.py

/Users/yamadayuuhei/Study/AIE/simplechat/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:276: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:81.)
  cpu = _conversion_method_template(device=torch.device("cpu"))
Traceback (most recent call last):
  File "/Users/yamadayuuhei/Study/AIE/lecture-ai-engineering/day1/03_FastAPI/app.py", line 3, in <module>
    from transformers import pipeline
ModuleNotFoundError: No module named 'transformers'


FastAPIが起動すると、APIとクライアントが通信するためのURL（エンドポイント）が作られます。

URLが作られるのと合わせて、Swagger UIというWebインターフェースが作られます。

Swagger UIにアクセスすることで、APIの仕様を確認できたり、APIをテストすることができます。

Swagger UIを利用することで、APIを通してLLMを動かしてみましょう。

後片付けとして、使う必要のないngrokのトンネルを削除します。

In [ ]:
from pyngrok import ngrok
ngrok.kill()